# Researcher Chat
Hello, everyone. The purpose of this notebook is to create a chatbot that reads papers online and answers technical questions from  Arxiv or SciHub.

You can download one of the best models of Generative AI that are  free and open source.
or you can connect with your WatsonX account or ChatGPT account.

The idea of this program is to develop a simple tool that helps researchers find insights about issues that we have in science.



### First we need to load our libraries needed for this task

In [1]:
import urllib.request
import fitz
import re
import numpy as np
import tensorflow_hub as hub
import openai
import gradio as gr
import os
from sklearn.neighbors import NearestNeighbors

C:\Users\rusla\.conda\envs\textgen\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def download_pdf(url, output_path):
    urllib.request.urlretrieve(url, output_path)
def preprocess(text):
    text = text.replace('\n', ' ')
    text = re.sub('\s+', ' ', text)
    return text
def pdf_to_text(path, start_page=1, end_page=None):
    doc = fitz.open(path)
    total_pages = doc.page_count

    if end_page is None:
        end_page = total_pages

    text_list = []

    for i in range(start_page-1, end_page):
        text = doc.load_page(i).get_text("text")
        text = preprocess(text)
        text_list.append(text)

    doc.close()
    return text_list
def text_to_chunks(texts, word_length=150, start_page=1):
    text_toks = [t.split(' ') for t in texts]
    page_nums = []
    chunks = []
    
    for idx, words in enumerate(text_toks):
        for i in range(0, len(words), word_length):
            chunk = words[i:i+word_length]
            if (i+word_length) > len(words) and (len(chunk) < word_length) and (
                len(text_toks) != (idx+1)):
                text_toks[idx+1] = chunk + text_toks[idx+1]
                continue
            chunk = ' '.join(chunk).strip()
            chunk = f'[Page no. {idx+start_page}]' + ' ' + '"' + chunk + '"'
            chunks.append(chunk)
    return chunks

In [3]:
class SemanticSearch:
    def __init__(self):
        self.use = hub.load('https://tfhub.dev/google/universal-sentence-encoder/4')
        self.fitted = False
    def fit(self, data, batch=1000, n_neighbors=5):
        self.data = data
        self.embeddings = self.get_text_embedding(data, batch=batch)
        n_neighbors = min(n_neighbors, len(self.embeddings))
        self.nn = NearestNeighbors(n_neighbors=n_neighbors)
        self.nn.fit(self.embeddings)
        self.fitted = True
    def __call__(self, text, return_data=True):
        inp_emb = self.use([text])
        neighbors = self.nn.kneighbors(inp_emb, return_distance=False)[0]
        if return_data:
            return [self.data[i] for i in neighbors]
        else:
            return neighbors
    def get_text_embedding(self, texts, batch=1000):
        embeddings = []
        for i in range(0, len(texts), batch):
            text_batch = texts[i:(i+batch)]
            emb_batch = self.use(text_batch)
            embeddings.append(emb_batch)
        embeddings = np.vstack(embeddings)
        return embeddings

In [4]:
def load_recommender(path, start_page=1):
    global recommender
    texts = pdf_to_text(path, start_page=start_page)
    chunks = text_to_chunks(texts, start_page=start_page)
    recommender.fit(chunks)
    return 'Corpus Loaded.'

In [5]:
def generate_text_openai(openAI_key,prompt, engine="text-davinci-003"):
    openai.api_key = openAI_key
    completions = openai.Completion.create(
        engine=engine,
        prompt=prompt,
        max_tokens=512,
        n=1,
        stop=None,
        temperature=0.7,
    )
    message = completions.choices[0].text
    return message

In [6]:
def query(query,modelo="flan-t5-small",tokens=100000):
    options=""
    tok_len=tokens
    t5query = f"""Question: "{query}" Context: {options}"""        
    if (modelo=="flan-t5-small" or modelo=="flan-t5-large"):
       tokenizer = AutoTokenizer.from_pretrained("google/{}".format(modelo))
       model = AutoModelForSeq2SeqLM.from_pretrained("google/{}".format(modelo)).to(device)
       inputs = tokenizer(t5query, return_tensors="pt").to(device)
       outputs = model.generate(**inputs, max_new_tokens=tok_len)
    else:
        model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-125M").to(device)
        tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125M")
        input_ids = tokenizer(t5query, return_tensors="pt").to(device) 
        outputs = model.generate(**input_ids, do_sample=True, max_length=tok_len) 
    generation=tokenizer.batch_decode(outputs, skip_special_tokens=True)
    
    return '\n'.join(generation)

In [7]:
question='What is the capital of Italy'

In [8]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM
import torch
import gradio as gr
import re
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
query(question, modelo ="flan-t5-small" ,tokens=100)

bin C:\Users\rusla\.conda\envs\textgen\lib\site-packages\bitsandbytes\libbitsandbytes_cuda117.dll


'rome'

In [10]:
def generate_answer(question,openAI_key):
    topn_chunks = recommender(question)
    prompt = ""
    prompt += 'search results:\n\n'
    for c in topn_chunks:
        prompt += c + '\n\n'
        
    prompt += "Instructions: Compose a comprehensive reply to the query using the search results given. "\
              "Cite each reference using [ Page Number] notation (every result has this number at the beginning). "\
              "Citation should be done at the end of each sentence. If the search results mention multiple subjects "\
              "with the same name, create separate answers for each. Only include information found in the results and "\
              "don't add any additional information. Make sure the answer is correct and don't output false content. "\
              "If the text does not relate to the query, simply state 'Found Nothing'. Ignore outlier "\
              "search results which has nothing to do with the question. Only answer what is asked. The "\
              "answer should be short and concise. \n\nQuery: {question}\nAnswer: "
    
    prompt += f"Query: {question}\nAnswer:"
    answer = prompt
    #answer = generate_text(openAI_key, prompt,"text-davinci-003")
    return answer

In [11]:
def question_answer(url, file, question,openAI_key):
    if openAI_key.strip()=='':
        return '[ERROR]: Please enter you Open AI Key. Get your key here : https://platform.openai.com/account/api-keys'
    if url.strip() == '' and file == None:
        return '[ERROR]: Both URL and PDF is empty. Provide atleast one.'
    
    if url.strip() != '' and file != None:
        return '[ERROR]: Both URL and PDF is provided. Please provide only one (eiter URL or PDF).'

    if url.strip() != '':
        glob_url = url
        download_pdf(glob_url, 'corpus.pdf')
        load_recommender('corpus.pdf')

    else:
        old_file_name = file.name
        file_name = file.name
        file_name = file_name[:-12] + file_name[-4:]
        os.rename(old_file_name, file_name)
        load_recommender(file_name)

    if question.strip() == '':
        return '[ERROR]: Question field is empty'

    return generate_answer(question,openAI_key)

In [12]:
recommender = SemanticSearch()
title = 'PDF GPT'
description = """ PDF GPT allows you to chat with your PDF file using Universal Sentence Encoder and Open AI. It gives hallucination free response than other tools as the embeddings are better than OpenAI. The returned response can even cite the page number in square brackets([]) where the information is located, adding credibility to the responses and helping to locate pertinent information quickly."""

In [13]:
with gr.Blocks() as demo:

    gr.Markdown(f'<center><h1>{title}</h1></center>')
    gr.Markdown(description)

    with gr.Row():
        
        with gr.Group():
            gr.Markdown(f'<p style="text-align:center">Get your Open AI API key <a href="https://platform.openai.com/account/api-keys">here</a></p>')
            openAI_key=gr.Textbox(label='Enter your OpenAI API key here')
            url = gr.Textbox(label='Enter PDF URL here')
            gr.Markdown("<center><h4>OR<h4></center>")
            file = gr.File(label='Upload your PDF/ Research Paper / Book here', file_types=['.pdf'])
            question = gr.Textbox(label='Enter your question here')
            btn = gr.Button(value='Submit')
            btn.style(full_width=True)
#openai.api_key = os.getenv('Your_Key_Here') 
        with gr.Group():
            answer = gr.Textbox(label='The answer to your question is :')

        btn.click(question_answer, inputs=[url, file, question,openAI_key], outputs=[answer])

demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [14]:
question="Give me a summary of the abstract"
def generate_prompt(question):
    topn_chunks = recommender(question)
    results = ""
    results += 'search results:\n\n'
    for c in topn_chunks:
        results += c + '. \n\n'
        
    instruction = "Instructions: Compose a comprehensive reply to the query using the search results given. "\
              "Cite each reference using [ Page Number] notation (every result has this number at the beginning). "\
              "Citation should be done at the end of each sentence. If the search results mention multiple subjects "\
              "with the same name, create separate answers for each. Only include information found in the results and "\
              "don't add any additional information. Make sure the answer is correct and don't output false content. "\
              "If the text does not relate to the query, simply state 'Found Nothing'. Ignore outlier "\
              "search results which has nothing to do with the question. Only answer what is asked. The "\
              "answer should be short and concise."
    
    prompt = instruction + "\n\nQuery: {}".format(question) + " \n\n" + results + " \nAnswer:"
    
    #answer = generate_text(openAI_key, prompt,"text-davinci-003")
    #return answer
    return prompt


In [15]:
generate_prompt(question)

AttributeError: 'SemanticSearch' object has no attribute 'nn'

In [69]:
import arxiv
search = arxiv.Search(
  query = "Shell model",
  max_results = 2,
  sort_by = arxiv.SortCriterion.SubmittedDate
)

for result in search.results():
  print(result.title)

Learning Disentangled Avatars with Hybrid 3D Representations
Attention De-sparsification Matters: Inducing Diversity in Digital Pathology Representation Learning


In [85]:
titles = [result.title for result in search.results()]
authors = [result.authors for result in search.results()]
summary = [result.summary for result in search.results()]
entry_id = [result.entry_id for result in search.results()]
#download_source = [result.download_source for result in search.results()]
#download_pdf = [result.download_pdf for result in search.results()]
pdf_url = [result.pdf_url for result in search.results()]
categories = [result.categories for result in search.results()]
comment = [result.comment for result in search.results()]
doi = [result.doi for result in search.results()]
published = [result.published for result in search.results()]

In [71]:
import pandas as pd

In [90]:
df = pd.DataFrame({
    'title': titles,
    'authors': authors,
    'summary': summary,
    #'entry_id': entry_id,
    'pdf_url':pdf_url,
    'categories':categories,
    'published':published
    
})

In [91]:
df.insert(0, 'id', range(1, len(df) + 1))

In [92]:
print(f'{df.shape[0]} results were found.')

2 results were found.


In [93]:
df

,id,title,authors,summary,pdf_url,categories,published
0,1,Learning Disentangled Avatars with Hybrid 3D R...,"[Yao Feng, Weiyang Liu, Timo Bolkart, Jinlong ...",Tremendous efforts have been made to learn ani...,http://arxiv.org/pdf/2309.06441v1,"[cs.CV, cs.AI, cs.GR]",2023-09-12 17:59:36+00:00
1,2,Attention De-sparsification Matters: Inducing ...,"[Saarthak Kapse, Srijan Das, Jingwei Zhang, Ra...","We propose DiRL, a Diversity-inducing Represen...",http://arxiv.org/pdf/2309.06439v1,[cs.CV],2023-09-12 17:59:10+00:00


In [94]:
# python program to check if a path exists
#if path doesn’t exist we create a new path
from pathlib import Path
#creating a new directory called pythondirectory
Path("./papers").mkdir(parents=True, exist_ok=True)

In [77]:
# Method 2 to download pdfs
id=1
for result in search.results():
    if len(df)!=0:
        result.download_pdf(filename="./papers/downloaded-paper-{}.pdf".format(id))
        id=id+1
    else:
        print('No results were found.')

In [117]:
# Method 2 to download pdfs
from pathlib import Path
import requests
id=1
for link in pdf_url:
    print(link)
    if len(pdf_url)!=0:
        filename_="./papers/downloaded-paper-{}.pdf".format(id)
        filename = Path(filename_)
        url = link
        response = requests.get(url)
        filename.write_bytes(response.content)
        id=id+1
    else:
        print('No results were found.')

http://arxiv.org/pdf/2309.06441v1
http://arxiv.org/pdf/2309.06439v1


In [118]:
# Method 3 to download pdfs
import sys
import pyxiv
for arxiv_id in entry_id:
    arxiv_id=arxiv_id.replace("http://arxiv.org/abs/","")
    print(arxiv_id)
    if len(entry_id)!=0:
        eprint = pyxiv.eprint_from_arxiv_id(arxiv_id)
        eprint.download("./papers")
    else:
        print('No results were found.')

2309.06441v1
2309.06439v1


In [ ]:
pyxiv.ePrint.download?

In [147]:
#dir(eprint)

In [ ]:
Notes
https://pypi.org/project/scidownl/
http://lukasschwab.me/arxiv.py/index.html#example-fetching-results-with-a-custom-client
https://github.com/blairbilodeau/arxiv-biorxiv-search/blob/master/arxiv_search_walkthrough.ipynb
https://github.com/Tachyu/Arxiv-download
https://github.com/tom2rd/arXivSearchPython
https://github.com/enriquecallejascastro/arXiv-scrape/blob/main/arXiv-scrape.ipynb
https://github.com/enriquecallejascastro/arXiv-scrape/blob/main/arXiv-scrape.ipynb
https://github.com/MorenoLaQuatra/arxiv_search
https://huggingface.co/spaces/Deci/DeciLM-6b-instruct

In [149]:
#paper = next(arxiv.Search(id_list=["1605.08386v1"]).results())
# Download the PDF to the PWD with a default filename.
#paper.download_pdf()

In [150]:
# Download the PDF to the PWD with a default filename.
#result.download_pdf()